# Polars Manipulations On A Postgresql DataBase

## Imports

In [1]:
import polars as pl
import psycopg
from sqlalchemy import create_engine

## Secrets

### Retriving Postgresql Docker Secrets

In [2]:
db_secrets = {
    "user": "",
    "password": "",
    "name": ""
}

In [3]:
for key in db_secrets.keys():
    with open(f"./secrets/db_{key}.txt", "r") as file:
        db_secrets[key] = file.read().strip()

## Engine

### Engine URL

In [4]:
engine_url = f"postgresql+psycopg://{db_secrets['user']}:" + \
    f"{db_secrets['password']}@localhost/{db_secrets['name']}"

### Engine Creation

In [5]:
engine = create_engine(engine_url)

## Defining a DataFrame

In [6]:
valley_data = [
    pl.Series("users", ["Jhon Galt", "Francisco D'Anconia"], dtype=pl.String),
    pl.Series("ages", [45, 44], dtype=pl.UInt8),
    pl.Series("professions", ["engineer", "businessman"], dtype=pl.Categorical),
    pl.Series("weights", [180.32, 174.63], dtype=pl.Float32)
]

In [7]:
valley_df = pl.DataFrame(valley_data)

pl.Config.set_tbl_hide_column_data_types(True).set_tbl_hide_dataframe_shape(True)(float_precision=2)
valley_df

users,ages,professions,weights
"""Jhon Galt""",45,"""engineer""",180.32
"""Francisco D'Anconia""",44,"""businessman""",174.63


## Writing Manipulations

### Overwriting A DataBase Table

In [8]:
valley_df.write_database(
    table_name="valley_personnel",
    connection=engine_url,
    if_table_exists="replace"
)

-1

### Appending On A DataBase Table

#### Appending DataFrame

In [9]:
new_valley_data = [
    pl.Series("users", ["Dagny Taggart", "Henry Rearden"], dtype=pl.String),
    pl.Series("ages", [40, 48], dtype=pl.UInt8),
    pl.Series("professions", ["engineer", "businessman"], dtype=pl.Categorical),
    pl.Series("weights", [160.32, 190.63], dtype=pl.Float32)
]

In [10]:
append_df = pl.DataFrame(new_valley_data)

In [11]:
append_df.write_database(
    table_name="valley_personnel",
    connection=engine_url,
    if_table_exists="append"
)

-1

## Reading Manipulations

In [12]:
with engine.connect() as conn:
    result_df = pl.read_database("SELECT * FROM valley_personnel", conn)

result_df

users,ages,professions,weights
"""Jhon Galt""",45,"""engineer""",180.32
"""Francisco D'Anconia""",44,"""businessman""",174.63
"""Dagny Taggart""",40,"""engineer""",160.32
"""Henry Rearden""",48,"""businessman""",190.63
